In [27]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle

In [28]:
data = pd.read_csv('../yield.csv')
data = data.drop(columns = ['crop'], axis = 1)

In [29]:
data.head()

,Temperature,Moisture,Rainfall,PH,Nitrogen,Phosphorous,Potassium,Carbon,Soil_Acidic Soil,Soil_Alkaline Soil,Soil_Loamy Soil,Soil_Neutral Soil,Soil_Peaty Soil,Crop_Adzuki Beans,Crop_Black gram,Crop_Chickpea,Crop_Coconut,Crop_Coffee,Crop_Cotton,Crop_Ground Nut,Crop_Jute,Crop_Kidney Beans,Crop_Lentil,Crop_Moth Beans,Crop_Mung Bean,Crop_Peas,Crop_Pigeon Peas,Crop_Rubber,Crop_Sugarcane,Crop_Tea,Crop_Tobacco,Crop_apple,Crop_banana,Crop_grapes,Crop_maize,Crop_mango,Crop_millet,Crop_muskmelon,Crop_orange,Crop_papaya,Crop_pomegranate,Crop_rice,Crop_watermelon,Crop_wheat,synthetic_yield,yield_ton_per_acre
0,24.363503,37.554486,143.410205,6.498681,109.499747,13.855626,95.721685,3.125085,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,28.95,12.00
1,38.767858,43.140496,133.228024,7.486987,27.676799,9.113059,68.893868,0.386737,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.62,9.37
2,33.299849,69.818318,94.038478,6.750667,51.995954,90.628793,144.674779,3.138850,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.49,9.37
3,29.966462,33.800307,201.816668,4.833210,99.492096,144.917448,32.846768,0.630409,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.17,9.37
4,18.900466,70.875478,169.156040,5.242321,72.313402,75.408196,88.178462,3.609652,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,32.28,9.09


In [30]:
data.select_dtypes(exclude = ['int64', 'float64']).columns

Index([], dtype='object')

In [31]:
X = data.drop(columns = ['yield_ton_per_acre'], axis = 1)
y = data['yield_ton_per_acre']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

## XGBoost

In [32]:
'''from sklearn.model_selection import GridSearchCV

xgb_model = XGBRegressor()

param_grid = {
    'estimator__n_estimators': [100, 200, 300],  
    'estimator__learning_rate': [0.01, 0.05, 0.1],  
    'estimator__max_depth': [3, 4, 5, 6],  
    'estimator__subsample': [0.7, 0.8, 0.9, 1.0],  
    'estimator__colsample_bytree': [0.7, 0.8, 0.9, 1.0],  
}

grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, pred))
print('Optimized RMSE:', rmse)

print("Best Parameters found: ", grid_search.best_params_)'''

'from sklearn.model_selection import GridSearchCV\n\nxgb_model = XGBRegressor()\n\nparam_grid = {\n    \'estimator__n_estimators\': [100, 200, 300],  \n    \'estimator__learning_rate\': [0.01, 0.05, 0.1],  \n    \'estimator__max_depth\': [3, 4, 5, 6],  \n    \'estimator__subsample\': [0.7, 0.8, 0.9, 1.0],  \n    \'estimator__colsample_bytree\': [0.7, 0.8, 0.9, 1.0],  \n}\n\ngrid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring=\'neg_mean_squared_error\', n_jobs=-1, verbose=1)\ngrid_search.fit(X_train, y_train)\n\nbest_model = grid_search.best_estimator_\n\npred = best_model.predict(X_test)\n\nrmse = np.sqrt(mean_squared_error(y_test, pred))\nprint(\'Optimized RMSE:\', rmse)\n\nprint("Best Parameters found: ", grid_search.best_params_)'

In [33]:
'''with open('../models/yield_xg_model.pkl', 'wb') as xg_pkl:
    pickle.dump(best_model, xg_pkl)'''

"with open('../models/yield_xg_model.pkl', 'wb') as xg_pkl:\n    pickle.dump(best_model, xg_pkl)"

In [34]:
'''with open('../models/yield_xg_model.pkl', 'rb') as model_pkl:
    model = pickle.load(model_pkl)

    print(type(model))
'''

"with open('../models/yield_xg_model.pkl', 'rb') as model_pkl:\n    model = pickle.load(model_pkl)\n\n    print(type(model))\n"

## LightGBM

In [35]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
import joblib

lgb_model = LGBMRegressor(objective='regression', random_state=42)

param_dist = {
    'estimator__n_estimators': [100, 200, 300, 500],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__max_depth': [-1, 3, 5, 7, 10],  
    'estimator__num_leaves': [31, 50, 70, 100],  
    'estimator__subsample': [0.7, 0.8, 0.9, 1.0],  
    'estimator__colsample_bytree': [0.7, 0.8, 0.9, 1.0],  
}

random_search_lgb = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_dist,
    n_iter=50,  
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search_lgb.fit(X_train, y_train)

best_lgb_model = random_search_lgb.best_estimator_

preds_lgb = best_lgb_model.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(y_test, preds_lgb))
print("Optimized RMSE (LightGBM):", rmse_lgb)
print("Best Params (LightGBM):", random_search_lgb.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[LightGBM] [Warning] Unknown parameter: estimator__learning_rate
[LightGBM] [Warning] Unknown parameter: estimator__subsample
[LightGBM] [Warning] Unknown parameter: estimator__colsample_bytree
[LightGBM] [Warning] Unknown parameter: estimator__max_depth
[LightGBM] [Warning] Unknown parameter: estimator__n_estimators
[LightGBM] [Warning] Unknown parameter: estimator__num_leaves
[LightGBM] [Warning] Unknown parameter: estimator__learning_rate
[LightGBM] [Warning] Unknown parameter: estimator__subsample
[LightGBM] [Warning] Unknown parameter: estimator__colsample_bytree
[LightGBM] [Warning] Unknown parameter: estimator__max_depth
[LightGBM] [Warning] Unknown parameter: estimator__n_estimators
[LightGBM] [Warning] Unknown parameter: estimator__num_leaves
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: estimator__learning_rate
[LightGBM] [Warning] Unknown pa

In [36]:
with open('../models/yield_lgb_model.pkl', 'wb') as lgb_pkl:
    pickle.dump(best_lgb_model, lgb_pkl)

In [37]:
with open('../models/yield_lgb_model.pkl', 'rb') as model_pkl:
    model = pickle.load(model_pkl)

    print(type(model))


<class 'lightgbm.sklearn.LGBMRegressor'>
